In [1]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime, date
import csv
import time

In [2]:
# link = 'https://www.twse.com.tw/exchangeReport/BWIBBU_d?response=open_data'
# data = pd.read_csv(link, encoding='utf_8_sig')
# data.to_csv(r"D:\Stock\vol_select\twse_BWIBBU_d.csv", encoding='utf_8_sig')

In [3]:
def string_price_to_int(x):
    return int(x.replace(",", ""))

def string_price_to_float(x):
    try:
        return float(x)
    except:
        return 0
    
def vol_for_twse(x):
    try:
        return float(float(x)/1000)
    except:
        return 0

def moving_average(x, w):
    return np.convolve(x, np.ones(w), "valid") / w

def get_stock_volumn_price(yy, mm, dd, stock_tag):
    date_tag = date_tag = str(yy) + str(mm).zfill(2) + "01"
    link = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=open_data&date=%s&stockNo=%s'%(date_tag, stock_tag)
    try:
        df = pd.read_csv(link, encoding='utf_8_sig')
    except:
        return None
    df_target = df[['日期', '成交股數', '收盤價']]
    df_target.iloc[:, 1] = df_target.iloc[:, 1].apply(string_price_to_int)
    df_target.iloc[:, 2] = df_target.iloc[:, 2].apply(vol_for_twse)
    return df_target

def get_otc_stock_volumn_price(yy, mm, dd, stock_tag):
    yy = yy - 1911
    json_data = requests.get('http://www.tpex.org.tw/web/stock/aftertrading/daily_trading_info/st43_result.php?d=%s/%s/%s&stkno=%s'%(yy, mm, dd, stock_tag)).json()
    columns = ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']
    df = pd.DataFrame(json_data['aaData'], columns=columns)
    df_target = df[['日期', '成交股數', '收盤價']]
    df_target.iloc[:, 1] = df_target.iloc[:, 1].apply(string_price_to_float)
    df_target.iloc[:, 2] = df_target.iloc[:, 2].apply(string_price_to_float)
    return df_target

In [20]:
def get_time_duration_stock_info(time_list, stock_tag, min_volumn = 150, ma_num = 5, isOtc = False):
    df_all = pd.DataFrame()
    for time_data in time_list:
        yy = time_data[0]
        for mm in range(time_data[1], time_data[2] + 1):
            if isOtc:
                df = get_otc_stock_volumn_price(yy, mm, "01", stock_tag)
            else:
                df = get_stock_volumn_price(yy, mm, "01", stock_tag)
                if df is None:
                    continue
                time.sleep(0.15)
            df_all = pd.concat([df_all, df], axis=0)
            
    df_np = df_all.to_numpy().copy()
    
    if len(df_np) == 0:
        return None, None
    if df_np[:, 1].max() > min_volumn:
        df_np[:, 1][df_np[:, 1] < min_volumn] = min_volumn
    if df_np[:, 1].max() == df_np[:, 1].min():
        return None, None
    if df_np[:, 2].max() == df_np[:, 2].min():
        return None, None
    
    df_np[:, 1] = (df_np[:, 1] - df_np[:, 1].min()) / ((df_np[:, 1].max() - df_np[:, 1].min()))
    # df_np[:, 2] = (df_np[:, 2] - df_np[:, 2].min()) / ((df_np[:, 2].max() - df_np[:, 2].min()))
    
    # df_vol_ma = moving_average(df_np[:, 1], ma_num)
    # df_pri_ma = moving_average(df_np[:, 2], ma_num)
    df_vol_ma = df_np[:, 1]
    df_pri_ma = df_np[:, 2]
    
    vol_data = df_vol_ma
    pri_data = df_pri_ma
    return vol_data, pri_data

def calculate_target(vol_data, pri_data):
    aa = -7
    bb = -30
    if vol_data[bb:aa].mean() == 0:
        return False
    if pri_data[bb:aa].mean() == 0:
        return False
    try:
        vol_valid = vol_data[aa:].mean() / vol_data[bb:aa].mean() > 5
        previous_high_valid = pri_data[-10:].max() / pri_data[-1] <= 1.08
        # previous_high_valid = previous_high_valid & (pri_data[-20:].max() / pri_data[-1] < 1.05)
        # pri_valid = pri_data[aa:].mean() / pri_data[bb:aa].mean() > 1.03
        # pri_ma = moving_average(pri_data, 4)
        # trend_valid =  pri_ma[-8:].mean()/pri_ma[-88:-80].mean() > 0.8
        pri_5ma = moving_average(pri_data, 5)
        pri_10ma = moving_average(pri_data, 10)
        pri_20ma = moving_average(pri_data, 20)
        pri_60ma = moving_average(pri_data, 60)
        trend_valid = (pri_5ma[-1]/pri_10ma[-1] > 0.98) and (pri_5ma[-1]/pri_20ma[-1] > 1.03) and (pri_5ma[-1]/pri_60ma[-1] > 1.04)
        ma_pos_valid = (pri_data[-1] / pri_5ma[-1] >= 0.95) and (pri_data[-1] / pri_10ma[-1] >= 0.98) and (pri_data[-1] / pri_20ma[-1] >= 1.03) and (pri_data[-1] / pri_60ma[-1] >= 1.04)
    except:
        return False
    return vol_valid and previous_high_valid and trend_valid and ma_pos_valid

In [23]:
stock_list = []
data_path = r"D:\Stock\vol_select\data\twse.csv"
isOtc = False
# data_path = r"D:\Stock\vol_select\data\otc.csv"
# isOtc = True
with open(data_path, newline='', encoding='utf_8_sig') as csvfile:
    line_list = csv.reader(csvfile)
    for line in line_list:
        stock_list.append(line[0])

In [24]:
target_list = []
time_list = [
    [2023, 12, 12],
    [2024,  1,  3],
    # [2023,  7,  10],
]

for stock in stock_list:
    vol_data, pri_data = get_time_duration_stock_info(time_list, stock, min_volumn=150, ma_num=1, isOtc=isOtc)
    if vol_data is None or pri_data is None:
        continue
    valid = calculate_target(vol_data, pri_data)
    if valid:
        target_list.append(stock)
        print(stock)

1439
1587
1603
1608
1612
2328
2704
3058
3062
4540
4562
6215
6753
8033


In [ ]:
1220
1439
1587
1597
1603
1608
1612
1617
2328
2354
2359
2453
2464
2505
2704
3059
3062
3706
4540
4562
4576
4583
6166
6191
6215
6443
6605
6753
8033
8374

In [ ]:
1220
1439
1587
1597
1603
1608
1612
1617
2328
2354
2359
2453
2464
2505
2704
3059
3062
3706
4540
4562
4576
4583
4755
5469
6166
6191
6215
6443
6605
6753
8033
8374

In [ ]:
1220
1439
1587
1597
1603
1608
1612
1617
2010
2236
2323
2328
2354
2359
2453
2464
2505
2704
3062
3704
3706
4540
4576
4583
4938
5222
6128
6166
6191
6215
6605
6753
8033
8374

2070
3628
4580
4728
5201
5543
6171
6227
6294
6577
6609
6752
6859
8074
8433

In [ ]:
6752 6859 8109 6171
2505 2704

In [ ]:
1220
1439
1587
1597
1603
1605
1608
1609
1612
1615
1616
1617
1723
2010
2236
2323
2354
2359
2453
2464
2505
2704
2889
3704
3706
4540
4576
4583
4938
5222
6128
6166
6215
6605
6753
8033
8374
8443


2070
3426
3628
4580
5489
5543
6151
6171
6227
6294
6514
6523
6577
6609
6752
6859
8109
8433
8941

In [ ]:
1220
1439
1442
1457
1587
1597
1605
1608
1612
1615
1616
1723
2010
2236
2317
2323
2354
2359
2453
2464
2471
2611
2704
2889
3706
4540
4576
4583
4938
4958
5222
5534
6128
6166
6215
6605
6753
8374
8443

2070
3628
4580
4728
5310
5489
5543
6021
6151
6227
6523
6577
6609
6761
8433

In [ ]:
2070
c:\Python37\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: Mean of empty slice.
c:\Python37\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in longlong_scalars
  ret = ret / rcount
c:\Python37\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: Mean of empty slice.
c:\Python37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice.
c:\Python37\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: Mean of empty slice.
c:\Python37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in double_scalars
4506
4530
4554
4580
4584
5543
6151
c:\Python37\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in double_scalars
6904
8921

In [ ]:
1439
1587
1597
1612
c:\Python37\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in double_scalars
2374
2453
3059
3706
4576
c:\Python37\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: Mean of empty slice.
c:\Python37\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in longlong_scalars
  ret = ret / rcount
c:\Python37\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: Mean of empty slice.
c:\Python37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice.
c:\Python37\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: Mean of empty slice.
5469
6166
6215
6443
6753
8033
8374

In [ ]:
1457
1470
1587
c:\Python37\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars
1597
1608
2009
2317
2323
2354
2496
2597
2611
2834
c:\Python37\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: Mean of empty slice.
c:\Python37\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: Mean of empty slice.
c:\Python37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice.
c:\Python37\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: Mean of empty slice.
6128
6206
6230
6278
6605
6753
8374
8443
9902